In [5]:

import torch
import torch.nn as nn
import torch.nn.functional as F

nclasses = 43 # GTSRB as 43 classes

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, kernel_size=1)
        self.conv2 = nn.Conv2d(1, 29, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv3 = nn.Conv2d(29, 59, kernel_size=3)
        self.maxpool3 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv4 = nn.Conv2d(59, 74, kernel_size=3)
        self.maxpool4 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv2_drop = nn.Dropout2d()
        self.conv3_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1184, 300)
        self.fc2 = nn.Linear(300, nclasses)
        self.conv0_bn = nn.BatchNorm2d(3)
        self.conv1_bn = nn.BatchNorm2d(1)
        self.conv2_bn = nn.BatchNorm2d(29)
        self.conv3_bn = nn.BatchNorm2d(59)
        self.conv4_bn = nn.BatchNorm2d(74)
        self.dense1_bn = nn.BatchNorm1d(300)
    def forward(self, x):
        x =  F.relu(self.conv1_bn(self.conv1(self.conv0_bn(x))))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3( self.maxpool2(x))))
        x = F.relu(self.conv4_bn(self.conv4( self.maxpool3(x))))
        x = self.maxpool4(x)        
        x = x.view(-1, 1184)
        x = F.relu(self.fc1(x))
        x = self.dense1_bn(x)
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.dropout(x, training=self.training)
        return F.log_softmax(x)
        


In [6]:
from __future__ import print_function
import argparse
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


batch_size =50                                                                                                                                                                                     
epochs = 10000                                                                                                                                                                                                                                                                                                                                                                                 
seed = 1                                                                                                                                                                                           
log_interval=180                                                                                                                                                                                   
data = "Dataset"                                                                                                                                                                                                                                                                                                                                                                                          
torch.manual_seed(1)                                                                                                                                                                               
lr =0.007                                                                                                                                                                                          
momentum = 0.8                                                                                                                                                                                     
decay = 0.9                                                                                                                                                                                        
step = 1000                                                                                                                                                                                        
l2_norm = 0.00001  
cuda = True
resume = False
# These may change as described in paper

In [9]:
from data import initialize_data, data_transforms 
initialize_data("Dataset") # extracts the zip files, makes a validation set

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(data + '/train_images',
                         transform=data_transforms),
    batch_size=batch_size, shuffle=True, num_workers=1)




val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(data + '/val_images',
                         transform=data_transforms),
    batch_size=batch_size, shuffle=False, num_workers=1)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net()
if  cuda: 
    model.to(device)

if resume :
    state_dict = torch.load("model_78.pth")
    model.load_state_dict(state_dict) 



Dataset/val_images not found, making a validation set
Dataset


In [21]:
           
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        data =data.to(device)
        target =target.to(device)
        output = model(data)
        validation_loss += F.nll_loss(output, target, size_average=False).data.item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    return validation_loss



def train(epoch , train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target).cuda()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.data.item()))



In [22]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr , momentum=momentum, weight_decay=l2_norm, nesterov=True)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay)

temp = 10
for epoch in range(1, epochs):
    train(epoch, train_loader)
    val = validation()
    if epoch % step :
        scheduler.step()
    if val < temp : 
        temp = val
        model_file = 'model_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py ' + model_file + '` to generate the Kaggle formatted csv file')
    


<ipython-input-5-493c3bbaedf2>:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/35339 (0%)]	Loss: 2.048301
Train Epoch: 1 [9000/35339 (25%)]	Loss: 1.959192
Train Epoch: 1 [18000/35339 (51%)]	Loss: 1.615918
Train Epoch: 1 [27000/35339 (76%)]	Loss: 1.871183


<ipython-input-21-b7733b96ade7>:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Validation set: Average loss: 1.6684, Accuracy: 2776/3870 (72%)


Saved model to model_1.pth. You can run `python evaluate.py model_1.pth` to generate the Kaggle formatted csv file
Train Epoch: 2 [0/35339 (0%)]	Loss: 1.482977
Train Epoch: 2 [9000/35339 (25%)]	Loss: 1.649883
Train Epoch: 2 [18000/35339 (51%)]	Loss: 1.544524
Train Epoch: 2 [27000/35339 (76%)]	Loss: 1.780353

Validation set: Average loss: 1.0703, Accuracy: 3422/3870 (88%)


Saved model to model_2.pth. You can run `python evaluate.py model_2.pth` to generate the Kaggle formatted csv file
Train Epoch: 3 [0/35339 (0%)]	Loss: 1.535221
Train Epoch: 3 [9000/35339 (25%)]	Loss: 1.743137
Train Epoch: 3 [18000/35339 (51%)]	Loss: 1.610816
Train Epoch: 3 [27000/35339 (76%)]	Loss: 1.466971

Validation set: Average loss: 0.9065, Accuracy: 3398/3870 (88%)


Saved model to model_3.pth. You can run `python evaluate.py model_3.pth` to generate the Kaggle formatted csv file
Train Epoch: 4 [0/35339 (0%)]	Loss: 1.799325
Train Epoch: 4 [9000/

Train Epoch: 26 [18000/35339 (51%)]	Loss: 1.631186
Train Epoch: 26 [27000/35339 (76%)]	Loss: 1.763516

Validation set: Average loss: 0.2094, Accuracy: 3780/3870 (98%)

Train Epoch: 27 [0/35339 (0%)]	Loss: 1.539320
Train Epoch: 27 [9000/35339 (25%)]	Loss: 1.366191
Train Epoch: 27 [18000/35339 (51%)]	Loss: 1.646657
Train Epoch: 27 [27000/35339 (76%)]	Loss: 1.421237

Validation set: Average loss: 0.1970, Accuracy: 3792/3870 (98%)


Saved model to model_27.pth. You can run `python evaluate.py model_27.pth` to generate the Kaggle formatted csv file
Train Epoch: 28 [0/35339 (0%)]	Loss: 1.671547
Train Epoch: 28 [9000/35339 (25%)]	Loss: 1.645420
Train Epoch: 28 [18000/35339 (51%)]	Loss: 1.421773
Train Epoch: 28 [27000/35339 (76%)]	Loss: 1.385467

Validation set: Average loss: 0.1986, Accuracy: 3784/3870 (98%)

Train Epoch: 29 [0/35339 (0%)]	Loss: 1.212677
Train Epoch: 29 [9000/35339 (25%)]	Loss: 1.436485
Train Epoch: 29 [18000/35339 (51%)]	Loss: 1.309268
Train Epoch: 29 [27000/35339 (76%)]	Los

Train Epoch: 56 [27000/35339 (76%)]	Loss: 1.294185

Validation set: Average loss: 0.1902, Accuracy: 3786/3870 (98%)

Train Epoch: 57 [0/35339 (0%)]	Loss: 1.635070
Train Epoch: 57 [9000/35339 (25%)]	Loss: 1.493790
Train Epoch: 57 [18000/35339 (51%)]	Loss: 1.411140
Train Epoch: 57 [27000/35339 (76%)]	Loss: 1.571375

Validation set: Average loss: 0.1926, Accuracy: 3795/3870 (98%)

Train Epoch: 58 [0/35339 (0%)]	Loss: 1.757313
Train Epoch: 58 [9000/35339 (25%)]	Loss: 1.812798
Train Epoch: 58 [18000/35339 (51%)]	Loss: 1.438956
Train Epoch: 58 [27000/35339 (76%)]	Loss: 1.627344

Validation set: Average loss: 0.1941, Accuracy: 3790/3870 (98%)

Train Epoch: 59 [0/35339 (0%)]	Loss: 1.367664
Train Epoch: 59 [9000/35339 (25%)]	Loss: 1.574591
Train Epoch: 59 [18000/35339 (51%)]	Loss: 1.844805
Train Epoch: 59 [27000/35339 (76%)]	Loss: 1.488171

Validation set: Average loss: 0.1875, Accuracy: 3796/3870 (98%)


Saved model to model_59.pth. You can run `python evaluate.py model_59.pth` to generate the

Train Epoch: 85 [18000/35339 (51%)]	Loss: 1.572524
Train Epoch: 85 [27000/35339 (76%)]	Loss: 1.446008

Validation set: Average loss: 0.1902, Accuracy: 3794/3870 (98%)

Train Epoch: 86 [0/35339 (0%)]	Loss: 1.528356
Train Epoch: 86 [9000/35339 (25%)]	Loss: 1.721123
Train Epoch: 86 [18000/35339 (51%)]	Loss: 1.310812
Train Epoch: 86 [27000/35339 (76%)]	Loss: 1.691008

Validation set: Average loss: 0.1927, Accuracy: 3793/3870 (98%)

Train Epoch: 87 [0/35339 (0%)]	Loss: 1.438344
Train Epoch: 87 [9000/35339 (25%)]	Loss: 1.315058
Train Epoch: 87 [18000/35339 (51%)]	Loss: 1.638368
Train Epoch: 87 [27000/35339 (76%)]	Loss: 1.228431

Validation set: Average loss: 0.1844, Accuracy: 3793/3870 (98%)

Train Epoch: 88 [0/35339 (0%)]	Loss: 1.749081
Train Epoch: 88 [9000/35339 (25%)]	Loss: 1.987643
Train Epoch: 88 [18000/35339 (51%)]	Loss: 1.527516
Train Epoch: 88 [27000/35339 (76%)]	Loss: 1.455897

Validation set: Average loss: 0.1896, Accuracy: 3791/3870 (98%)

Train Epoch: 89 [0/35339 (0%)]	Loss: 1.7

Train Epoch: 116 [9000/35339 (25%)]	Loss: 1.529236
Train Epoch: 116 [18000/35339 (51%)]	Loss: 1.569811
Train Epoch: 116 [27000/35339 (76%)]	Loss: 1.620784

Validation set: Average loss: 0.1935, Accuracy: 3790/3870 (98%)

Train Epoch: 117 [0/35339 (0%)]	Loss: 1.363202
Train Epoch: 117 [9000/35339 (25%)]	Loss: 1.719801
Train Epoch: 117 [18000/35339 (51%)]	Loss: 2.119694
Train Epoch: 117 [27000/35339 (76%)]	Loss: 1.437583

Validation set: Average loss: 0.1862, Accuracy: 3792/3870 (98%)

Train Epoch: 118 [0/35339 (0%)]	Loss: 1.318840
Train Epoch: 118 [9000/35339 (25%)]	Loss: 1.310902
Train Epoch: 118 [18000/35339 (51%)]	Loss: 1.461570
Train Epoch: 118 [27000/35339 (76%)]	Loss: 1.564135

Validation set: Average loss: 0.1905, Accuracy: 3789/3870 (98%)

Train Epoch: 119 [0/35339 (0%)]	Loss: 1.418877
Train Epoch: 119 [9000/35339 (25%)]	Loss: 2.117791
Train Epoch: 119 [18000/35339 (51%)]	Loss: 1.500465
Train Epoch: 119 [27000/35339 (76%)]	Loss: 1.613297

Validation set: Average loss: 0.1891, Ac

KeyboardInterrupt: 